In [1]:
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

#for search capabilites import podaacpy
import podaac.podaac as podaac
import podaac.podaac_utils as putil
# then create an instance of the Podaac class
p = podaac.Podaac()

url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds_usv = xr.open_dataset(url)
ds_usv

ds_usv2 = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'}).rename({'longitude':'lon','latitude':'lat'})
ds_usv_subset = ds_usv2.sel(time=slice('2018-04-12T02','2018-06-10T18')) 
start_time=pd.to_datetime(str(ds_usv2.time.min().data)).strftime('%Y-%m-%dT%H:%m:%SZ') 
end_time=pd.to_datetime(str(ds_usv2.time.max().data)).strftime('%Y-%m-%dT%H:%m:%SZ') 
ds_usv_subset
print('start: ',start_time,'end: ',end_time)

#dataset_id = 'PODAAC-GHGMR-4FJ04'  #MUR SST looked up on podaac website
dataset_id = 'PODAAC-GHOST-4FK01'  #smaller data
dataset_id = 'PODAAC-GHGDM-4FD02' #DMI
gresult = p.granule_search(dataset_id=dataset_id,
                           start_time=start_time,
                           end_time=end_time,
                           items_per_page='100')
urls = putil.PodaacUtils.mine_opendap_urls_from_granule_search(gresult)
urls = [w[:-5] for w in urls]  #remove html from urls

ds_sst = xr.open_mfdataset(urls[0:3],coords='minimal')
ds_sst.time

start:  2018-04-11T18:04:00Z end:  2018-06-11T05:06:00Z


<xarray.DataArray 'time' (time: 3)>
array(['1970-01-02T18:19:00.000000000', '1970-01-02T18:19:09.000000000',
       '1970-01-02T18:19:17.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1970-01-02T18:19:00 ... 1970-01-02T18:19:17
Attributes:
    standard_name:  time
    long_name:      reference time of sst field
    axis:           T
    comment:        Nominal time of Level 4 analysis

In [4]:
ds_sst.time

<xarray.DataArray 'time' (time: 3)>
array(['2018-04-12T09:00:00.000000000', '2018-04-13T09:00:00.000000000',
       '2018-04-14T09:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-04-12T09:00:00 ... 2018-04-14T09:00:00
Attributes:
    long_name:      reference time of sst field
    standard_name:  time
    axis:           T
    comment:        Nominal time of analyzed fields

In [ ]:
ds_sst = xr.open_dataset('https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2011/236/20110824090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc') #,coords='minimal')
ds_sst.time